In [1]:

pip install idx2numpy

  Preparing metadata (setup.py) ... done
  Created wheel for idx2numpy: filename=idx2numpy-1.2.3-py3-none-any.whl size=7904 sha256=9479e92823972349c8e96fe1cdc124c31644daabdb6a8b0505371c8241fe2c03
  Stored in directory: /root/.cache/pip/wheels/e0/f4/e7/643fc5f932ec2ff92997f43f007660feb23f948aa8486f1107
Successfully built idx2numpy
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
# import torchvision
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
import idx2numpy
import time
from sklearn.metrics import accuracy_score, f1_score

In [3]:

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
print(device)


cuda


In [5]:
import os

# List all files and directories in /kaggle/working/
print(os.listdir('/kaggle/working/'))

# If you used "Add Data" in Kaggle, list files in the `/kaggle/input/` directory
print(os.listdir('/kaggle/input/'))

['.virtual_documents']
['mnist-dataset']


In [6]:
# Check if the dataset directory exists
dataset_dir = '/kaggle/input/mnist-dataset/'
print(os.listdir(dataset_dir))

['t10k-labels-idx1-ubyte', 'train-images.idx3-ubyte', 't10k-images-idx3-ubyte', 't10k-labels.idx1-ubyte', 't10k-images.idx3-ubyte', 'train-labels.idx1-ubyte', 'train-labels-idx1-ubyte', 'train-images-idx3-ubyte']


In [7]:
import idx2numpy
import os

# Check if the dataset directory exists
dataset_dir = '/kaggle/input/mnist-dataset/'
print(os.listdir(dataset_dir))

# Update paths with the correct filenames
train_images_path = os.path.join(dataset_dir, 'train-images-idx3-ubyte/train-images-idx3-ubyte')
train_labels_path = os.path.join(dataset_dir, 'train-labels-idx1-ubyte/train-labels-idx1-ubyte')
test_images_path = os.path.join(dataset_dir, 't10k-images-idx3-ubyte/t10k-images-idx3-ubyte')
test_labels_path = os.path.join(dataset_dir, 't10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte')

# Load data into NumPy arrays
train_images = idx2numpy.convert_from_file(train_images_path)
train_labels = idx2numpy.convert_from_file(train_labels_path)
test_images = idx2numpy.convert_from_file(test_images_path)
test_labels = idx2numpy.convert_from_file(test_labels_path)

# Check the shapes of loaded data
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

['t10k-labels-idx1-ubyte', 'train-images.idx3-ubyte', 't10k-images-idx3-ubyte', 't10k-labels.idx1-ubyte', 't10k-images.idx3-ubyte', 'train-labels.idx1-ubyte', 'train-labels-idx1-ubyte', 'train-images-idx3-ubyte']
(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [8]:
print(train_images.shape)

(60000, 28, 28)


In [9]:

train_images.max()

255

In [10]:
# Normalisation des données
train_images = train_images / 255.0
test_images = test_images / 255.0

In [11]:
print(train_images.max())
print(train_images.min())

1.0
0.0


In [12]:
# Transformations pour les datasets
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),  # Redimensionnement à 224x224
    transforms.ToTensor()
])

In [13]:
#custionm mnist dataset to vgg
class mnist_dataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform


    def __len__(self):
        return len(self.images)

   
    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [14]:
train_dataset = mnist_dataset(train_images, train_labels, transform=transform)
test_dataset = mnist_dataset(test_images, test_labels, transform=transform)

In [15]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [16]:
vgg16 = models.vgg16(weights=models.VGG16_Weights.DEFAULT)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 208MB/s]  


In [17]:
vgg16.features[0] = torch.nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=0)

In [18]:
num_classes = 10
vgg16.classifier[6] = torch.nn.Linear(vgg16.classifier[6].in_features, num_classes)
vgg16.features = torch.nn.Sequential(*list(vgg16.features.children())[:-1])

In [19]:
vgg16.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=Fals

In [20]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(vgg16.parameters(), lr=0.01, momentum=0.9)

In [21]:
# Entraînement du modèle
num_epochs = 5
start_time = time.time()

In [22]:
for epoch in range(num_epochs):
    vgg16.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = vgg16(images)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}")

training_time = time.time() - start_time
print(f"Training Time: {training_time:.2f} seconds")

Epoch 1/5, Loss: 0.1267
Epoch 2/5, Loss: 0.0258
Epoch 3/5, Loss: 0.0174
Epoch 4/5, Loss: 0.0139
Epoch 5/5, Loss: 0.0102
Training Time: 4265.38 seconds


In [23]:
# Évaluation
vgg16.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = vgg16(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calcul des métriques
accuracy = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average="weighted")

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.9921
F1 Score: 0.9921
